In [155]:
import yfinance as yf
# https://pypi.org/project/yfinance/
import pandas as pd
from datetime import datetime, timedelta, date
import time
# from reader import feed
from pandas.tseries.frequencies import to_offset
import numpy as np
# from pandas.tseries.offsets import BDay

In [156]:
tic = time.perf_counter()

today = date.today()
print("Today's date:", today)
days_back = 365*3
rows_drop = days_back-2
start_date = (date.today() - timedelta(days=days_back)).strftime('%Y-%m-%d')
start_date

Today's date: 2021-08-13


'2018-08-14'

In [157]:
# tickers = pd.read_csv("Input/tickers_input.csv")
# tickers = "SPY AAPL MSFT"
# tickers = ['AAPL','MSFT','AMD']
tickers = ['AAPL']
tickers

['AAPL']

In [158]:
# df = yf.download(tickers,start= start_date, end=date.today())
# df.head()

logic = {'Open'  : 'first',
         'High'  : 'max',
         'Low'   : 'min',
         'Close' : 'last',
         'Volume': 'sum'}

dfout = pd.DataFrame()

for ticker in tickers:
    df = yf.Ticker(ticker)
    df = df.history('3y')
#     df.to_csv('Output/df.csv')  # used to test is data is being collected correctly
    df.drop(columns=['Dividends','Stock Splits'],inplace=True)

In [159]:
    df1w = yf.download(ticker,period='3y',interval='1wk')
#     df1w = df.history('3y')
    df1w = df1w.dropna(how='any')
#     df1w.to_csv('Output/df1w.csv')  # used to test is data is being collected correctly
#     df.drop(columns=['Dividends','Stock Splits'],inplace=True)

    df3d = yf.download(ticker,period='3y',interval='1d')
#     df1w = df.history('3y')
    df3d = df3d.dropna(how='any')
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint
    
    df3d = df.resample('3B').apply(logic)
#     df3d.index = df3d.index + to_offset('3B')
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint
    
    df['conv_line'] = ((df.High.rolling(9).max())+(df.Low.rolling(9).min()))/2 #checked
    df['base_line'] = ((df.High.rolling(26).max())+(df.Low.rolling(26).min()))/2 #checked
    df['lead_a'] = ((df['conv_line']+df['base_line'])/2).shift(25) # checked
    df['lead_b'] = (((df.High.rolling(52).max())+(df.Low.rolling(52).min()))/2).shift(25) # checked
    df['lag_span'] = df.Close.shift(-25) #checked
    df['crit1'] = (df.conv_line>df.base_line).astype(int)  # conversion line above baseline, checked
    df['crit2'] = (df.lead_a.shift(-25)>df.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    df['crit3'] = (df.Close>df.lead_a).astype(int)  # price above Lead 1, checked
    df['crit4'] = (df.Close>df.lead_b).astype(int)  # price above Lead 2, checked
    df['crit5'] = (df.lead_a.shift(-25)>df.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    df['crit6'] = (((df.lag_span>df.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    df['crit7'] = (df.lead_b.shift(-25)>df.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df['test2'] = df.lead_a.shift(-25)-df.lead_a.shift(-24)  #  this code used to check crit2
    # df['test3'] = df.Close - df.lead_a # used to check crit3
    # df['test4'] = df.Close - df.lead_b # used to check crit4
    # df['test5'] = df.lead_a.shift(-25)-df.lead_b.shift(-25)  # test for crit5
    # df['test6'] = (df.lag_span - df.High).shift(25) # test for crit6
    # df['test7'] = df.lead_b.shift(-25)-df.lead_b.shift(-24)  #  this code used to check crit7
#     df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
#     df.to_csv('Output/df.csv')  # data checkpoint

#     this method doesn't line up exactly with TradingView but
#     is needed to merge the daily and 3 day timeframes data
    x = 3
    df['op'] = df.Open  # checked
    df['hi'] = df.High.rolling(x).max().shift(1-x) # checked, avg dev = -0.1% +/- 0.4%
    df['lo'] = df.Low.rolling(x).min().shift(1-x) # checked, avg dev = 0.1% +/- 0.6%
    df['clse'] = df.Close.shift(1-x) # checked, avg dev = 0.0% +/- 1.1%
    df['vol'] = (df.Volume.shift(1-x) + df.Volume.shift(2-x) + df.Volume) # checked, avg dev = -2.8% +/- 10.0%
    df['conv_line2'] = (((df.hi.rolling(9*x).max())+(df.lo.rolling(9*x).min()))/2).shift(1-x) # checked, avg dev = -2.8% +/- 10.0%
    df['base_line2'] = (((df.hi.rolling(26*x).max())+(df.lo.rolling(26*x).min()))/2).shift(1-x) #checked, avg dev = -0.1% +/- 1.1%
    df['lead_a2'] = ((df['conv_line2']+df['base_line2'])/2).shift(25*x) # checked, avg dev = 0.2% +/- 1.0%
    df['lead_b2'] = (((df.hi.rolling(52*x).max())+(df.lo.rolling(52*x).min()))/2).shift(25*x) # checked, , avg dev = 0.9% +/- 1.5%
    df['lag_span2'] = df.clse.shift(-25*x) #checked, avg dev = -0.3% +/- 4.1%
    df['crit12'] = (df.conv_line2>df.base_line2).astype(int)  # conversion line above baseline, checked
    df['crit22'] = (df.lead_a2.shift(-25*x)>df.lead_a2.shift(-24*x)).astype(int)  # lead 1 moving up, checked
    df['crit32'] = (df.clse>df.lead_a2).astype(int)  # price above Lead 1, checked
    df['crit42'] = (df.clse>df.lead_b2).astype(int)  # price above Lead 2, checked
    df['crit52'] = (df.lead_a2.shift(-25*x)>df.lead_b2.shift(-25*x)).astype(int)  # lead 1 above lead 2, checked
    df['crit62'] = (((df.lag_span2>df.hi).shift(25*x)).fillna(0).astype(int)) # lag line above price action, checked
    df['crit72'] = (df.lead_b2.shift(-25*x)>df.lead_b2.shift(-24*x)).astype(int)  # lead 2 moving up, checked
    # df3d['test2'] = df3d.lead_a.shift(-25)-df3d.lead_a.shift(-24)  #  this code used to check crit2
    # df3d['test3'] = df3d.Close - df3d.lead_a # used to check crit3
    # df3d['test4'] = df3d.Close - df3d.lead_b # used to check crit4
    # df3d['test5'] = df3d.lead_a.shift(-25)-df3d.lead_b.shift(-25)  # test for crit5
    # df3d['test6'] = (df3d.lag_span - df3d.High).shift(25) # test for crit6
    # df3d['test7'] = df3d.lead_b.shift(-25)-df3d.lead_b.shift(-24)  #  this code used to check crit7
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint
    df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
    df = df.drop(df.columns[[7,8,9,10,11,12,13,14,15,16]],axis=1, inplace=False)  #remove unnecessary columns
    df.to_csv('Output/df3d2.csv')  # data checkpoint

# this section is accurate for 3day timeframe and used to verify results of calculated 3 day timeframe
#     df3d['conv_line'] = ((df3d.High.rolling(9).max())+(df3d.Low.rolling(9).min()))/2 #checked
#     df3d['base_line'] = ((df3d.High.rolling(26).max())+(df3d.Low.rolling(26).min()))/2 #checked
#     df3d['lead_a'] = ((df3d['conv_line']+df3d['base_line'])/2).shift(25) # checked
#     df3d['lead_b'] = (((df3d.High.rolling(52).max())+(df3d.Low.rolling(52).min()))/2).shift(25) # checked
#     df3d['lag_span'] = df3d.Close.shift(-25) #checked
#     df3d['crit1'] = (df3d.conv_line>df3d.base_line).astype(int)  # conversion line above baseline, checked
#     df3d['crit2'] = (df3d.lead_a.shift(-25)>df3d.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
#     df3d['crit3'] = (df3d.Close>df3d.lead_a).astype(int)  # price above Lead 1, checked
#     df3d['crit4'] = (df3d.Close>df3d.lead_b).astype(int)  # price above Lead 2, checked
#     df3d['crit5'] = (df3d.lead_a.shift(-25)>df3d.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
#     df3d['crit6'] = (((df3d.lag_span>df3d.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
#     df3d['crit7'] = (df3d.lead_b.shift(-25)>df3d.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df3d['test2'] = df3d.lead_a.shift(-25)-df3d.lead_a.shift(-24)  #  this code used to check crit2
    # df3d['test3'] = df3d.Close - df3d.lead_a # used to check crit3
    # df3d['test4'] = df3d.Close - df3d.lead_b # used to check crit4
    # df3d['test5'] = df3d.lead_a.shift(-25)-df3d.lead_b.shift(-25)  # test for crit5
    # df3d['test6'] = (df3d.lag_span - df3d.High).shift(25) # test for crit6
    # df3d['test7'] = df3d.lead_b.shift(-25)-df3d.lead_b.shift(-24)  #  this code used to check crit7
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint
#     df3d = df3d.drop(df3d.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
#     df3d.to_csv('Output/df3d.csv')  # data checkpoint

    df1w['conv_line'] = ((df1w.High.rolling(9).max())+(df1w.Low.rolling(9).min()))/2 #checked
    df1w['base_line'] = ((df1w.High.rolling(26).max())+(df1w.Low.rolling(26).min()))/2 #checked
    df1w['lead_a'] = ((df1w['conv_line']+df1w['base_line'])/2).shift(25) # checked
    df1w['lead_b'] = (((df1w.High.rolling(52).max())+(df1w.Low.rolling(52).min()))/2).shift(25) # checked
    df1w['lag_span'] = df1w.Close.shift(-25) #checked
    df1w['crit1'] = (df1w.conv_line>df1w.base_line).astype(int)  # conversion line above baseline, checked
    df1w['crit2'] = (df1w.lead_a.shift(-25)>df1w.lead_a.shift(-24)).astype(int)  # lead 1 moving up, checked
    df1w['crit3'] = (df1w.Close>df1w.lead_a).astype(int)  # price above Lead 1, checked
    df1w['crit4'] = (df1w.Close>df1w.lead_b).astype(int)  # price above Lead 2, checked
    df1w['crit5'] = (df1w.lead_a.shift(-25)>df1w.lead_b.shift(-25)).astype(int)  # lead 1 above lead 2, checked
    df1w['crit6'] = (((df1w.lag_span>df1w.High).shift(25)).fillna(0).astype(int)) # lag line above price action, checked
    df1w['crit7'] = (df1w.lead_b.shift(-25)>df1w.lead_b.shift(-24)).astype(int)  # lead 2 moving up, checked
    # df1w['test2'] = df1w.lead_a.shift(-25)-df1w.lead_a.shift(-24)  #  this code used to check crit2
    # df1w['test3'] = df1w.Close - df1w.lead_a # used to check crit3
    # df1w['test4'] = df1w.Close - df1w.lead_b # used to check crit4
    # df1w['test5'] = df1w.lead_a.shift(-25)-df1w.lead_b.shift(-25)  # test for crit5
    # df1w['test6'] = (df1w.lag_span - df1w.High).shift(25) # test for crit6
    # df1w['test7'] = df1w.lead_b.shift(-25)-df1w.lead_b.shift(-24)  #  this code used to check crit7
    df1w = df1w.drop(df1w.columns[[0,1,2,3,4,5,6,7,8,9,10]],axis=1, inplace=False)  #remove unnecessary columns
#     df1w.to_csv('Output/df1w.csv')  # data checkpoint
    
    df.reset_index(inplace=True)
#     df3d.reset_index(inplace=True)
    df1w.reset_index(inplace=True)
#     df1w.to_csv('Output/df1w.csv')  # data checkpoint

    merged_df = pd.merge(df1w,\
                         df,\
                         left_on=[df1w['Date'].dt.year, df1w['Date'].dt.strftime('%U')],\
                         right_on=[df['Date'].dt.year, df['Date'].dt.strftime('%U')],\
                         how='right',suffixes=('_1W','_1D'),).drop(['key_0','key_1'],axis=1)
    merged_df=merged_df.apply(lambda x: x.fillna(method='ffill') if x.dtype.kind in \
                              'biufc' else x.fillna(method='ffill'))
    merged_df.to_csv('Output/merged_df.csv')  # data checkpoint
    
#     merged_df2 = pd.merge(df,\
#                          df3d,\
#                          left_on=[df['Date'].dt.year, df['Date'].dt.strftime('3B')],\
#                          right_on=[df3d['Date'].dt.year, df3d['Date'].dt.strftime('3B')],\
#                          how='right',suffixes=('_1D','_3D'),).drop(['key_0','key_1'],axis=1)
#     merged_df2=merged_df2.apply(lambda x: x.fillna(method='ffill') if x.dtype.kind in \
#                               'biufc' else x.fillna(method='ffill'))
#     merged_df2.to_csv('Output/merged_df2.csv')  # data checkpoint

#     df = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]],axis=1, inplace=False)  #remove unnecessary columns
#     df['total_pts'] = df.sum(axis=1)  # adds up the points
#     df = df.drop(df.columns[[0,1,2,3,4,5,6]],axis=1,inplace=False) #remove unnecessary columns
#     row_count = len(df.index) #get the row count of the dataframe
#     df = df.iloc[row_count-2: , :] #remove all rows except the last two
#     chg = df.total_pts.diff() #find difference between the last two rows
#     df['chg'] = chg #add the chg data to the dataframe
#     row_count = len(df.index) #get the row count of the dataframe
#     df = df.iloc[row_count-1: , :] # remove all but the last row
#     df['ticker'] = ticker # add ticker label column
#     df = df[['ticker','total_pts','chg']] # move ticker column to the front

#     dfout = dfout.append(df)
    dfout = dfout.append(merged_df)

dfout.to_csv('Output/check_file.csv')
# print(ticker)
# pd.set_option('display.max_rows',999)
dfout.head(10)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date_1W,crit1_1W,crit2_1W,crit3_1W,crit4_1W,crit5_1W,crit6_1W,crit7_1W,Date_1D,crit1_1D,...,crit5_1D,crit6_1D,crit7_1D,crit12,crit22,crit32,crit42,crit52,crit62,crit72
0,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-14,0,...,0,0,0,0,0,0,0,0,0,0
1,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-15,0,...,0,0,0,0,0,0,0,0,0,0
2,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-16,0,...,0,0,0,0,0,0,0,0,0,0
3,2018-08-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-17,0,...,0,0,0,0,0,0,0,0,0,0
4,2018-08-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-20,0,...,0,0,0,0,0,0,0,0,0,0
5,2018-08-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-21,0,...,0,0,0,0,0,0,0,0,0,0
6,2018-08-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-22,0,...,0,0,0,0,0,0,0,0,0,0
7,2018-08-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-23,0,...,0,0,0,0,0,0,0,0,0,0
8,2018-08-20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-24,0,...,0,0,0,0,0,0,0,0,0,0
9,2018-08-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2018-08-27,0,...,0,0,0,0,0,0,0,0,0,0


In [160]:
toc = time.perf_counter()
print(f"Total processing time is {toc-tic:0.4f} seconds")
# count = len(tickers)

# # for i in count:
# df['50MA'] = tickers[0].Close.rolling(50).mean()
# df
# df.to_csv('check_file.csv')

Total processing time is 1.0680 seconds


In [161]:
#     counter = 1
#     row_count = len(df['Date'])
#     for i in range(row_count):
#         if df['Date'][counter] == df[]
        
# df.iloc[0,:]

# df['3D_date']=df['Date'].apply(lambda x: x if x==df3d['Date'].item() else 'nope')
# df

# for i in range(len(df['Date'])): # len() verified
#     if df3d['Date'][i] == df['Date'][i]:
#         df['Date_3D'] = df[df3d['Date'][i]]
    
# len(df['Date'])
# df.tail()

# df.loc[df['Date']==df3d['Date'],'date_match']=df3d['Date']
# df.loc[df['Date']!=df3d['Date'],'date_match']=''

# df